In [1]:
### If you need teh files again
#!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
#!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"

In [2]:
# Import required libraries
import pandas as pd
import jupyter_dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [3]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [4]:
# Create a dash application
app = jupyter_dash.JupyterDash(__name__)

In [5]:
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id = 'site-dropdown',
                                             options = [{'label': 'All Sites', 'value':'ALL'},
                                                        {'label': 'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                                        {'label': 'CCAFS SLC-40', 'value':'CCAFS SLC-40'},
                                                        {'label': 'KSC LC-39A', 'value':'KSC LC-39A'},
                                                        {'label': 'CCAFS SLC-40', 'value':'CCAFS SLC-40'},],
                                             value = 'ALL',
                                             placeholder = 'Select a Launch Site Here',
                                             searchable = True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id = 'payload-slider',
                                                min = 0,
                                                max = 10000,
                                                step = 1000,
                                                marks = {0 : '0',
                                                         1000: '1000',
                                                         2000: '2000',
                                                         3000: '3000',
                                                         4000: '4000',
                                                         5000: '5000',
                                                         6000: '6000',
                                                         7000: '7000',
                                                         8000: '8000',
                                                         9000: '9000',
                                                         10000:'10000',},
                                                value = [min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [6]:
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property = 'figure'),
              Input(component_id='site-dropdown', component_property = 'value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, 
                     values = 'class',
                     names = 'Launch Site',
                     title = 'Total Successful Launches by Site')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        pct_succ = spacex_df[spacex_df['Launch Site'] == entered_site].sum()['class']/spacex_df[spacex_df['Launch Site'] == entered_site].shape[0] * 100
        pct_fail = 100-pct_succ
        test = pd.DataFrame(data = {'Rate': ['Success Rate', 'Failure Rate'],'Number': [pct_succ, pct_fail]})
        fig = px.pie(test, 
               values = 'Number',
               names = 'Rate',
               title = 'Total Successful Launches at '  + entered_site)
        return fig
    

In [7]:
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property = 'figure'),
              Input(component_id='site-dropdown', component_property = 'value'),
              Input(component_id='payload-slider', component_property = 'value'))
def get_scatter_chart(entered_site, payload_mass):
    if entered_site == 'ALL':
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] > payload_mass[0]) & (spacex_df['Payload Mass (kg)'] < payload_mass[1])]
        fig = px.scatter(filtered_df, 
                         x = 'Payload Mass (kg)',
                         y = 'class',
                         color = 'Booster Version Category',
                         title = 'Correlation Between Payload and Success for All Sites')
        return fig
    else:
        filtered_df = spacex_df[(spacex_df['Launch Site'] == entered_site) & (spacex_df['Payload Mass (kg)'] > payload_mass[0]) & (spacex_df['Payload Mass (kg)'] < payload_mass[1])]
        fig = px.scatter(filtered_df, 
                         x = 'Payload Mass (kg)',
                         y = 'class',
                         color = 'Booster Version Category',
                         title = 'Correlation Between Payload and Success for ' + entered_site)
        return fig

In [9]:
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash app running on http://127.0.0.1:8050/
